In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import json
# import plotly.io as pio
# pio.renderers.default = "chrome"

In [2]:
# px.scatter(y=[np.nan]).show()

In [3]:
towns_df = pd.read_csv("communes-departement-region_nearest-station_duration-trip.csv", sep=",").reset_index(drop=True)
towns_df = towns_df[~towns_df["town_station_trip_duration"].isna()].reset_index(drop=True)
towns_df["code_commune_INSEE"] = towns_df["code_commune_INSEE"].astype("string")
towns_df["station_paris_trip_duration"] = pd.to_timedelta(towns_df["station_paris_trip_duration"])
towns_df["town_station_trip_duration"] = pd.to_timedelta(towns_df["town_station_trip_duration"])
towns_df["town_paris_trip_duration"] = towns_df["station_paris_trip_duration"]+towns_df["town_station_trip_duration"]
towns_df["town_paris_trip_duration_in_hours"] = towns_df["town_paris_trip_duration"]/pd.Timedelta(hours=1)

C:\Users\gcathelain\.conda\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
towns_df.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region,station_place_id,station_paris_trip_duration,town_station_trip_duration,town_paris_trip_duration,town_paris_trip_duration_in_hours
0,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes,ChIJO-m5bVtpi0cREilPAo5kQC0,0 days 04:15:00,0 days 00:20:49,0 days 04:35:49,4.596944
1,1009,ANDERT ET CONDON,1300,ANDERT ET CONDON,NaN,45.787357,5.657883,9.0,NaN,Andert-et-Condon,Andert-et-Condon,1,Ain,84.0,Auvergne-Rhône-Alpes,ChIJO-m5bVtpi0cREilPAo5kQC0,0 days 04:15:00,0 days 00:11:38,0 days 04:26:38,4.443889
2,1010,ANGLEFORT,1350,ANGLEFORT,NaN,45.909372,5.795160,10.0,NaN,Anglefort,Anglefort,1,Ain,84.0,Auvergne-Rhône-Alpes,ChIJDahVTqJ7i0cRugs-8Faxn8k,0 days 03:29:00,0 days 00:07:55,0 days 03:36:55,3.615278
3,1015,ARBOYS EN BUGEY,1300,ARBOYS EN BUGEY,ARBIGNIEU,45.723762,5.652631,15.0,NaN,Arboys en Bugey,Arboys en Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes,ChIJO-m5bVtpi0cREilPAo5kQC0,0 days 04:15:00,0 days 00:22:45,0 days 04:37:45,4.629167
4,1015,ARBOYS EN BUGEY,1300,ARBOYS EN BUGEY,ST BOIS,45.723762,5.652631,15.0,NaN,Arboys en Bugey,Arboys en Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes,ChIJO-m5bVtpi0cREilPAo5kQC0,0 days 04:15:00,0 days 00:22:45,0 days 04:37:45,4.629167


In [27]:
towns_df.loc[towns_df["nom_commune"]=="Lille",:]

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region,station_place_id,station_paris_trip_duration,town_station_trip_duration,town_paris_trip_duration,town_paris_trip_duration_in_hours


https://www.data.gouv.fr/fr/datasets/decoupage-administratif-communal-francais-issu-d-openstreetmap/

https://cadastre.data.gouv.fr/data/etalab-cadastre/latest/geojson/france

In [6]:
geojson_url = "https://github.com/gcathelain/isochrone-maps/blob/master/communes-20190101/communes-20190101.json"
fig = px.choropleth(towns_df, geojson=geojson_url, color = "town_paris_trip_duration_in_hours", locations="code_commune_INSEE", featureidkey="properties.insee")
fig.show()

In [ ]:
geojson_url = "https://github.com/gcathelain/isochrone-maps/blob/master/communes-20190101/communes-20190101.json"
fig = px.choropleth(towns_df.loc[:10,:], geojson=geojson_url, color = "town_paris_trip_duration_in_hours", locations="code_commune_INSEE", featureidkey="properties.insee")
fig.show()

In [5]:
with open(r"communes-20190101/communes-20190101.json", "r") as json_file:
    openstreetmap_data = json.load(json_file)

Très long à filtrer...

In [ ]:
openstreetmap_data["features"] = [feature for feature in openstreetmap_data["features"] if feature["properties"]["insee"] in list(towns_df["code_commune_INSEE"])]
# openstreetmap_data["features"] = [feature for feature in openstreetmap_data["features"] if feature["properties"]["insee"] in list(towns_df["code_commune_INSEE"])]

In [ ]:
len(openstreetmap_data['features'])

In [ ]:
fig = px.choropleth(towns_df[:100], geojson=openstreetmap_data, color = "station_paris_trip_duration", locations="code_commune_INSEE", featureidkey="properties.insee")
fig.show()